In [1]:
from tweepy.streaming import StreamListener
from tweepy.auth import OAuthHandler
from tweepy import Stream

from datetime import datetime
import json
import time

from pymongo import MongoClient
import pymongo

In [2]:
#pyenv
import os
from dotenv import load_dotenv
load_dotenv()
CONSUMER_KEY = os.environ.get('CONSUMER_KEY')
CONSUMER_SECRET = os.environ.get('CONSUMER_SECRET')
ACCESS_TOKEN = os.environ.get('ACCESS_TOKEN')
ACCESS_TOKEN_SECRET = os.environ.get('ACCESS_TOKEN_SECRET')

In [3]:
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
#api = tweepy.API(auth)
#api.update_status('tweepy + oauth!')

In [4]:
client = MongoClient('localhost', 27017)
db = client.twitterdb
col = db.tweets
raw_col = db.raw_data
keywords = ['bieber']

In [5]:
import threading
import time
import sys
import os
# Define a function for the thread
def count_time( threadName, delay, limit):
    count = 0
    while count < limit:
        time.sleep(delay)
        count += 1
        print ("%s: %s" % ( threadName, time.ctime(time.time()) )  )
    print('finished ', threadName)
    #os._exit(0)

In [6]:
class MyListener(StreamListener):
    def __init__(self):
        super().__init__()
        self.counter = 0
        self.limit = 100
        
        #threading.Thread(target = count_time,
            #args= ("Time Counter", 1, 5)
        #).start()
        
    def on_data(self,data):
        
        tweet = json.loads(data)

        created_at = tweet["created_at"]
        id_str = tweet["id_str"]
        text = tweet["text"]
        
        user_id = tweet["user"]["id_str"]
        user_name = tweet["user"]["name"]
        screen_name = tweet["user"]["screen_name"]
        user_created_at = tweet["user"]["created_at"]
        
        obj = {
            "message_created_at":time.mktime(time.strptime( created_at,"%a %b %d %H:%M:%S +0000 %Y" )),
            "message_id":id_str,
            "message_text":text,
            "author_user_id": user_id,
            "author_created_at": time.mktime(time.strptime( user_created_at,"%a %b %d %H:%M:%S +0000 %Y" )),
            "author_user_name": user_name,
            "author_screen_name": screen_name
        }
        
        print('🐦', end='')

        tweetind = col.insert_one(obj).inserted_id

        tweetind_2 = raw_col.insert_one( json.loads(json.dumps(tweet)) ).inserted_id
        
        """ #Tweet limitation counter
        self.counter += 1
        if self.counter < self.limit:
            return True
        else:
            return False
        """ 
        
        return True
    
    def on_error(self, status):
        print(status)
        return False

In [7]:
mylistener = MyListener()
mystream = Stream(auth, listener = mylistener)

In [8]:
print('start stream')
#mystream.filter(track=keywords)
mystream.filter(track = keywords, is_async = True)
#mystream.filter(track = keywords, async = True)

time.sleep(30)

mystream.disconnect()

print('close stream')

start stream
Time Counter: Sun May  5 15:06:21 2019
🐦🐦🐦🐦🐦🐦Time Counter: Sun May  5 15:06:22 2019
🐦🐦🐦🐦🐦🐦close stream
Time Counter: Sun May  5 15:06:23 2019
Time Counter: Sun May  5 15:06:24 2019
Time Counter: Sun May  5 15:06:25 2019
finished  Time Counter


In [ ]:
all_values = col.find().sort([("message_created_at",pymongo.ASCENDING), ("author_created_at",pymongo.ASCENDING)])

for x in all_values:
    print(x)

In [ ]:
import json
import csv
from bson.json_util import dumps

json_values = json.loads(dumps( col.find().sort([("message_created_at",pymongo.ASCENDING), ("author_created_at",pymongo.ASCENDING)]) ))

with open('output.tsv', 'w') as output_file:
    dw = csv.DictWriter(output_file, sorted( json_values[0].keys() ), delimiter='\t')
    dw.writeheader()
    dw.writerows(json_values)

In [ ]:
class LimitedListener(StreamListener):

    def __init__(self):
        super().__init__()
        self.counter = 0
        self.limit = 10

    def on_status(self, status):
        try:
            userid = status.user.id_str
            geo = str(status.coordinates)
            if geo != "None":
                print(userid + ',' + geo)
            else:
                print("No coordinates")
            self.counter += 1
            if self.counter < self.limit:
                return True
            else:
                twitterStream.disconnect()
        except BaseException as e:
            print('failed on_status,',str(e))
            time.sleep(5)

In [ ]:
limited_listener = LimitedListener()
limited_stream = Stream(auth, listener = limited_listener)
print('inicia limited stream')
limited_stream.filter(track=keywords)
print('fecha limited stream')